In [ ]:
!pip install datasets transformers shutup ipython-autotime evaluate shutup --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
dataset_file = 'dataset-filtered-renamed.csv'
model_anu = 'Intent-Classification-small'
tokenizer_anu = 'Intent-Classification-small'
dataset_anu = 'Intent-Classification-small'
per_device_train_batch_size=64
per_device_eval_batch_size=64
num_train_epochs=8
base_model = 'joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

In [ ]:
base_path = '/content/drive/MyDrive/ExcelR_GenAI_Practicals/1-Intent-Classification-small-transformers-main/dataset_folder/data_BALANCE_DATA-True/'

In [ ]:
dataset = load_dataset("csv", data_files=base_path+dataset_file)
dataset = dataset["train"].shuffle(seed=42)

In [ ]:
dataset = dataset.remove_columns('labels')
dataset = dataset.train_test_split(train_size=0.9, seed=42)
# Rename the default test split to validation
dataset["validation"] = dataset.pop("test")

In [ ]:
# Process to generate a label column

from datasets import Dataset, Value, ClassLabel, Features

# Creating a ClassLabel Object
df = dataset["train"].to_pandas()
labels = df['intent'].unique().tolist()
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['intent'])
    return example

dataset = dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'label'],
        num_rows: 684
    })
})

In [ ]:
dataset.push_to_hub(f"anuz123/{dataset_anu}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/anuz123/Intent-Classification-small/commit/9a1a83df4a71eb541b668d1bced06cf2cdde1601', commit_message='Upload dataset', commit_description='', oid='9a1a83df4a71eb541b668d1bced06cf2cdde1601', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Pre-trained
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)

outputs = pretrained_tokenizer(
    dataset["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print("vocab_size: ", len(pretrained_tokenizer))

Input IDs length: 2
Input chunk lengths: [6, 9]
vocab_size:  30522


In [ ]:
txt = "how are you"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

[101, 2129, 2024, 2017, 102]
['[CLS]', 'how', 'are', 'you', '[SEP]']


In [ ]:
# Training a new tokeniser
def get_training_corpus():
    batch_size = 1000
    return (
        dataset["train"][i : i + batch_size]["text"]
        for i in range(0, len(dataset["train"]), batch_size)
    )
training_corpus = get_training_corpus()

for data in get_training_corpus():
    print(len(data))

1000
1000
1000
1000
1000
1000
153


In [ ]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)
vocab_size = len(pretrained_tokenizer)
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

print(tokenizer.eos_token_id, ",", tokenizer.vocab_size)

txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print(tokens)
converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

None , 5
[2, 1, 1, 1, 3]
['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[SEP]']


In [ ]:
print("New tokenizer: ", len(tokenizer.tokenize(txt)))
print("Pretrained tokenizer: ", len(pretrained_tokenizer.tokenize(txt)))

New tokenizer:  3
Pretrained tokenizer:  3


In [ ]:
# Training the model
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizerFast, RobertaTokenizer
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification, \
    TFRobertaForSequenceClassification, TFAutoModel, AutoModel, AutoModelForSequenceClassification
from sklearn import preprocessing
from datasets import load_dataset

In [ ]:
dataset = load_dataset(f'anuz123/Intent-Classification-small')
dataset

DataFilesNotFoundError: No (supported) data files found in anuz123/Intent-Classification-small

In [ ]:
# Creating datacollector and evaluator
from transformers import DataCollatorWithPadding
import evaluate

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
def id2label_gen(id):
    return ClassLabels.int2str(id)

def label2id_gen(label):
    return ClassLabels.str2int(label)

total_labels = ClassLabels.num_classes

print(id2label_gen(3), label2id_gen(id2label_gen(3)),  "total_labels: ", total_labels)

covid cases 3 total_labels:  21


In [ ]:
id2label = {i: ClassLabels.int2str(i) for i in range(total_labels)}
label2id = {ClassLabels.int2str(i): i for i in range(total_labels)}
id2label, label2id

({0: 'goodbye',
  1: 'volume control',
  2: 'play games',
  3: 'covid cases',
  4: 'open website',
  5: 'tell me joke',
  6: 'play on youtube',
  7: 'places near me',
  8: 'greet and hello hi kind of things, general check in',
  9: 'asking time',
  10: 'asking date',
  11: 'tell me news',
  12: 'asking weather',
  13: 'download youtube video',
  14: 'what can you do',
  15: 'take screenshot',
  16: 'send email',
  17: 'i am bored',
  18: 'click photo',
  19: 'tell me about',
  20: 'send whatsapp message'},
 {'goodbye': 0,
  'volume control': 1,
  'play games': 2,
  'covid cases': 3,
  'open website': 4,
  'tell me joke': 5,
  'play on youtube': 6,
  'places near me': 7,
  'greet and hello hi kind of things, general check in': 8,
  'asking time': 9,
  'asking date': 10,
  'tell me news': 11,
  'asking weather': 12,
  'download youtube video': 13,
  'what can you do': 14,
  'take screenshot': 15,
  'send email': 16,
  'i am bored': 17,
  'click photo': 18,
  'tell me about': 19,
  'send 

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=total_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([60]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.weight: found shape torch.Size([60, 768]) in the checkpoint and torch.Size([21, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Create tokenized dataset

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6153 [00:00<?, ? examples/s]

Map:   0%|          | 0/684 [00:00<?, ? examples/s]

In [ ]:
# Finetuning the model
config = {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": id2label,
  "initializer_range": 0.02,
  "label2id": label2id,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": False,
  "tie_weights_": True,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(base_model)
config.update({"id2label": id2label})
config.update({"label2id": label2id})
# Push the config to your namespace with the name "my-finetuned-bert".
# config.push_to_hub(f"anuz123/{model_anu}")

In [ ]:
output_dir_model_hf_name=f"anuz123/{model_anu}"

training_args = TrainingArguments(
    output_dir=output_dir_model_hf_name,
    learning_rate=2e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.242064,0.331871
2,No log,1.561545,0.551170
3,No log,1.053547,0.738304


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(
    f"anuz123/{tokenizer_anu}")
model = AutoModelForSequenceClassification.from_pretrained(
    f"anuz123/{model_anu}")
classifier_return_all_scores = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True )
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
sequence = "make me laugh"

# Classify the sequence
results = classifier(sequence)

# Print the results
print(results)

[{'label': 'tell me joke', 'score': 0.9442085027694702}]
